In [1]:
import numpy as np
import pandas as pd

from sklearn.metrics import f1_score,confusion_matrix,classification_report,mean_absolute_percentage_error as mape
from lightgbm import LGBMClassifier

In [ ]:

data=pd.read_excel('/content/drive/MyDrive/Colab Notebooks/WeldRight Dataset.xlsx')
data=data.drop(["Machine",'Unnamed: 13', 'Unnamed: 14', 'Unnamed: 15', 'Unnamed: 16',
       'Unnamed: 17', 'Unnamed: 18', 'Unnamed: 19', 'Unnamed: 20',
       'Unnamed: 21', 'Unnamed: 22', 'Unnamed: 23', 'Unnamed: 24',
       'Unnamed: 25', 'Unnamed: 26', 'Unnamed: 27', 'Unnamed: 28',
       'Unnamed: 29', 'Unnamed: 30', 'Unnamed: 31', 'Unnamed: 32',
       'Unnamed: 33', 'Unnamed: 34', 'Unnamed: 35', 'Unnamed: 36',
       'Unnamed: 37', 'Unnamed: 38', 'Unnamed: 39', 'Unnamed: 40',
       'Unnamed: 41', 'Unnamed: 42', 'Unnamed: 43', 'Unnamed: 44',
       'Unnamed: 45', 'Unnamed: 46', 'Unnamed: 47', 'Unnamed: 48',
       'Unnamed: 49', 'Unnamed: 50', 'Unnamed: 51', 'Unnamed: 52',
       'Unnamed: 53', 'Unnamed: 54', 'Unnamed: 55', 'Unnamed: 56',
       'Unnamed: 57', 'Unnamed: 58', 'Unnamed: 59', 'Unnamed: 60',
       'Unnamed: 61', 'Unnamed: 62', 'Unnamed: 63', 'Unnamed: 64',
       'Unnamed: 65', 'Unnamed: 66', 'Unnamed: 67', 'Unnamed: 68',
       'Unnamed: 69', 'Unnamed: 70', 'Unnamed: 71', 'Unnamed: 72',
       'Unnamed: 73', 'Unnamed: 74', 'Unnamed: 75', 'Unnamed: 76',
       'Unnamed: 77', 'Unnamed: 78', 'Unnamed: 79', 'Unnamed: 80',
       'Unnamed: 81', 'Unnamed: 82', 'Unnamed: 83', 'Unnamed: 84',
       'Unnamed: 85', 'Unnamed: 86', 'Unnamed: 87'],axis=1)
data.columns

Index(['Employee Code', 'Production', 'Order Operation No', 'Date', 'Time',
       'Current', 'Humidity', 'Temperature', 'Flow', 'Job Temp', 'Voltage',
       'Defect'],
      dtype='object')

In [ ]:
df=data.copy()
df=df.drop(0)
df=df.drop(["Date","Time"],axis=1)

#encoding DEFECT column
df.Defect=df.Defect.replace({'No Defect':0, 'Tungsten Inclusion':1, 'No Defect73':0,
       'Tungsten Inclusion72':1, 'No Defect72':0, 'No Defect77':0,
       'No Defect79':0, 'No Defect80':0, 'Porosity':2, 'No Defect78':0,
       'No Defect76':0, 'No Defect0':0, 'No Defect82':0, 'No Defect85':0,
       'No Defect84':0, 'No Defect83':0, 'No Defect81':0, 'No Defect75':0,
       'No Defect86':0, 'Porosity86':2, 'No Defect87':0, 'Tungsten Inclusion78':1,
       'No Defect74':0, 'No Defect99':0, 'Porosity75':2, 'Tungsten Inclusion77':1,
       'No Defect71':0, 'No Defect40':0, 'Porosity80':2, 'No Defect70':0,
       'No Defect68':0, 'No Defect90':0, 'Tungsten Inclusion80':1,
       'No Defect69':0, 'No Defect67':0, 'No Defect63':0, 'No Defect64':0,
       'No Defect65':0, 'No Defect62':0, 'No Defect61':0, 'No Defect60':0,
       'No Defect59':0, 'Porosity76':2, 'No Defect66':0, 'Tungsten Inclusion71':1,
       'No Defect88':0, 'Tungsten Inclusion67':1, 'Tungsten Inclusion75':1,
       'Tungsten Inclusion76':1, 'Porosity71':2, 'No Defect58':0, 'No Defect57':0})

#encoding PRODUCTION column:
df.Production=df.Production.replace({'E15002965':1, 'E15002966':2 ,'-':4,
       15003062:5, 'E15003220':6, 'E15002881':7, 'E10002436':8, 'E50006711':9,
       'E15003200':10, 'E15003202':11, 'E15003056':12, 'E15002963':13, 'E10002437':14,
       'E15003219':15, 'E15003201':16, 'Test coopan':17, -15003217:18, -150033027:19})

#converting object type to numeric type
cols=['Employee Code', 'Order Operation No','Current',
       'Humidity', 'Temperature', 'Flow', 'Job Temp', 'Voltage']
df[cols] = df[cols].apply(pd.to_numeric, errors='coerce')

#imputing null values in PRODUCTION,ORDER OPERATION NUMBER,CURRENT
df.Production=df.Production.fillna(method="ffill")
df["Order Operation No"]=df["Order Operation No"].fillna(method="ffill")
df.Current=df.Current.fillna(method="ffill")

print(df.dtypes)
print("FINDING NULL COLUMNS:")
print(df.isnull().any())

df.head()

Employee Code           int64
Production            float64
Order Operation No    float64
Current               float64
Humidity                int64
Temperature             int64
Flow                  float64
Job Temp              float64
Voltage               float64
Defect                  int64
dtype: object
FINDING NULL COLUMNS:
Employee Code         False
Production            False
Order Operation No    False
Current               False
Humidity              False
Temperature           False
Flow                  False
Job Temp              False
Voltage               False
Defect                False
dtype: bool


,Employee Code,Production,Order Operation No,Current,Humidity,Temperature,Flow,Job Temp,Voltage,Defect
1,382617,1.0,240.0,1.13,74,23,0.01,29.3,15.2,0
2,382617,1.0,240.0,1.82,74,23,0.00,29.3,0.0,1
3,382617,1.0,240.0,2.91,74,23,0.00,29.3,0.0,0
4,382617,1.0,240.0,0.04,74,23,0.00,29.3,0.0,0
5,382617,1.0,240.0,2.51,74,23,0.00,27.8,0.0,0


## FEATURE ENGINEERING


In [ ]:
df1=df.copy()
df1["f2"]=np.log(df1.Voltage**0.9/df1["Job Temp"]**0.1)
df1["f1"]=np.log(df1.Voltage*0.9499-df1["Job Temp"]**0.1)+df1.f2-np.log10(df1.f2**25)
df1["f3"]=np.log(df1.Voltage*0.9499-df1["Job Temp"]**0.1) -np.log(df1.Current**0.004)
df1["f4"]=(df1["Job Temp"]<145).astype("int")
# df1["f5"]=
# corr = df1.corr()
# corr.style.background_gradient(cmap='coolwarm')
Y=df1["Defect"]
X=df1.drop(["Defect","Production","Order Operation No"],axis=1)
X=X.replace([np.inf, -np.inf], 0) 
X.f1=X.f1.fillna(method="ffill")
X.f2=X.f2.fillna(method="ffill")
X.f3=X.f2.fillna(method="ffill")
X.isnull().any()

/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Employee Code    False
Current          False
Humidity         False
Temperature      False
Flow             False
Job Temp         False
Voltage          False
f2               False
f1               False
f3               False
f4               False
dtype: bool

## TRAIN_TEST_SLPIT, BORDERLINE SMOTE RESAMPLING AND ML MODEL TRAINING

In [ ]:
#TRAIN_TEST SPLIT
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X,Y,random_state=1,test_size=0.1,stratify=Y)

print("y_train value counts:")
print(y_train.value_counts())
print("y_test value counts:")
print(y_test.value_counts())

#SMOTE RESAMPLING

from imblearn.over_sampling import BorderlineSMOTE
smote=BorderlineSMOTE(sampling_strategy="not majority",n_jobs=-1,k_neighbors=1,m_neighbors=2,kind="borderline-2",random_state=42)
X_sm,Y_sm=smote.fit_resample(x_train,y_train)
print(Y_sm.value_counts())

y_train value counts:
0    739602
1      4149
2      1029
Name: Defect, dtype: int64
y_test value counts:
0    82179
1      461
2      114
Name: Defect, dtype: int64
0    739602
1    739602
2    739602
Name: Defect, dtype: int64


THE MODEL BELOW IS ABLE TO PREDICT ALL THREE CLASSES

In [ ]:
model8=LGBMClassifier(colsample_bytree=0.5177908906249942,
               learning_rate=0.21034746012122046, max_bin=1023,
               min_child_samples=19, n_estimators=968, num_leaves=331,
               reg_alpha=0.026810451163601555, reg_lambda=0.026723380640834737,
               verbose=-1,n_jobs=-1)

model8.fit(X_sm,Y_sm)
pred=model8.predict(x_test)
print("mape:",mape(y_test,pred))
print("individual f1_score:",f1_score(y_test,pred,average=None))
print("weighted f1_score:",f1_score(y_test,pred,average="weighted"))
print("confusion_matrix:")
print(confusion_matrix(pred,y_test))
print("classification_report:")
print(classification_report(pred,y_test))

mape: 8380916241088.73
individual f1_score: [0.99651411 0.37799718 0.01481481]
weighted f1_score: 0.9917161578786122
confusion_matrix:
[[82045   327   113]
 [  114   134     0]
 [   20     0     1]]
classification_report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     82485
           1       0.29      0.54      0.38       248
           2       0.01      0.05      0.01        21

    accuracy                           0.99     82754
   macro avg       0.43      0.53      0.46     82754
weighted avg       1.00      0.99      0.99     82754



## SECOND MODEL WHICH CAN PREDICT TUNGSTEN DEFECT BETTER

In [ ]:
data2=data.copy()

data2=data2.drop(0)
data2=data2.drop(["Date","Time"],axis=1)

#encoding DEFECT column
data2.Defect=data2.Defect.replace({'No Defect':0, 'Tungsten Inclusion':1, 'No Defect73':0,
       'Tungsten Inclusion72':1, 'No Defect72':0, 'No Defect77':0,
       'No Defect79':0, 'No Defect80':0, 'Porosity':2, 'No Defect78':0,
       'No Defect76':0, 'No Defect0':0, 'No Defect82':0, 'No Defect85':0,
       'No Defect84':0, 'No Defect83':0, 'No Defect81':0, 'No Defect75':0,
       'No Defect86':0, 'Porosity86':2, 'No Defect87':0, 'Tungsten Inclusion78':1,
       'No Defect74':0, 'No Defect99':0, 'Porosity75':2, 'Tungsten Inclusion77':1,
       'No Defect71':0, 'No Defect40':0, 'Porosity80':2, 'No Defect70':0,
       'No Defect68':0, 'No Defect90':0, 'Tungsten Inclusion80':1,
       'No Defect69':0, 'No Defect67':0, 'No Defect63':0, 'No Defect64':0,
       'No Defect65':0, 'No Defect62':0, 'No Defect61':0, 'No Defect60':0,
       'No Defect59':0, 'Porosity76':2, 'No Defect66':0, 'Tungsten Inclusion71':1,
       'No Defect88':0, 'Tungsten Inclusion67':1, 'Tungsten Inclusion75':1,
       'Tungsten Inclusion76':1, 'Porosity71':2, 'No Defect58':0, 'No Defect57':0})

#encoding PRODUCTION column:
data2.Production=data2.Production.replace({'E15002965':1, 'E15002966':2 ,'-':4,
       15003062:5, 'E15003220':6, 'E15002881':7, 'E10002436':8, 'E50006711':9,
       'E15003200':10, 'E15003202':11, 'E15003056':12, 'E15002963':13, 'E10002437':14,
       'E15003219':15, 'E15003201':16, 'Test coopan':17, -15003217:18, -150033027:19})

#converting object type to numeric type
cols=['Employee Code', 'Order Operation No','Current',
       'Humidity', 'Temperature', 'Flow', 'Job Temp', 'Voltage']
data2[cols] = data2[cols].apply(pd.to_numeric, errors='coerce')

#imputing null values in PRODUCTION,ORDER OPERATION NUMBER,CURRENT
# data2.Production=data2.Production.fillna(method="ffill")
# data2["Order Operation No"]=data2["Order Operation No"].fillna(method="ffill")
# data2.Current=data2.Current.fillna(method="ffill")

#dropping the rows with missing values
data2=data2.dropna()
#CREATING TARGET AND FEATURE SETS
Y=data2["Defect"]
X=data2.drop(["Defect","Production","Order Operation No","Temperature","Flow"],axis=1)
print(np.unique(Y, return_counts=True))

print(data2.dtypes)
print("FINDING NULL COLUMNS:")
print(data2.isnull().any())

data2.head()

(array([0, 1, 2]), array([491853,   2817,    704]))
Employee Code           int64
Production            float64
Order Operation No    float64
Current               float64
Humidity                int64
Temperature             int64
Flow                  float64
Job Temp              float64
Voltage               float64
Defect                  int64
dtype: object
FINDING NULL COLUMNS:
Employee Code         False
Production            False
Order Operation No    False
Current               False
Humidity              False
Temperature           False
Flow                  False
Job Temp              False
Voltage               False
Defect                False
dtype: bool


,Employee Code,Production,Order Operation No,Current,Humidity,Temperature,Flow,Job Temp,Voltage,Defect
1,382617,1.0,240.0,1.13,74,23,0.01,29.3,15.2,0
2,382617,1.0,240.0,1.82,74,23,0.00,29.3,0.0,1
3,382617,1.0,240.0,2.91,74,23,0.00,29.3,0.0,0
4,382617,1.0,240.0,0.04,74,23,0.00,29.3,0.0,0
5,382617,1.0,240.0,2.51,74,23,0.00,27.8,0.0,0


In [ ]:
df_1=data2.copy()
df_1["f2"]=np.log(df_1.Voltage**0.9/df_1["Job Temp"]**0.1)
df_1["f1"]=np.log(df_1.Voltage*0.9499-df_1["Job Temp"]**0.1)+df_1.f2-np.log10(df_1.f2**25)
df_1["f3"]=np.log(df_1.Voltage*0.9499-df_1["Job Temp"]**0.1) -np.log(df_1.Current**0.004)
df_1["f4"]=(df_1["Job Temp"]<145).astype("int")
# df1["f5"]=
# corr = df1.corr()
# corr.style.background_gradient(cmap='coolwarm')
Y1=df_1["Defect"]
X1=df_1.drop(["Defect","Production","Order Operation No"],axis=1)
X1=X1.replace([np.inf, -np.inf], 0) 
X1.f1=X1.f1.fillna(method="ffill")
X1.f2=X1.f2.fillna(method="ffill")
X1.f3=X1.f2.fillna(method="ffill")
X1.isnull().any()

/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


Employee Code    False
Current          False
Humidity         False
Temperature      False
Flow             False
Job Temp         False
Voltage          False
f2               False
f1               False
f3               False
f4               False
dtype: bool

In [ ]:
#TRAIN_TEST SPLIT
from sklearn.model_selection import train_test_split
x_train,x_test,y_train,y_test=train_test_split(X1,Y1,random_state=1,test_size=0.1,stratify=Y1)

print("y_train value counts:")
print(y_train.value_counts())
print("y_test value counts:")
print(y_test.value_counts())

#SMOTE RESAMPLING

from imblearn.over_sampling import BorderlineSMOTE
smote=BorderlineSMOTE(sampling_strategy="not majority",n_jobs=-1,k_neighbors=1,m_neighbors=2,kind="borderline-2",random_state=42)
X_sm1,Y_sm1=smote.fit_resample(x_train,y_train)
print(Y_sm1.value_counts())

y_train value counts:
0    442667
1      2535
2       634
Name: Defect, dtype: int64
y_test value counts:
0    49186
1      282
2       70
Name: Defect, dtype: int64
0    442667
2    442667
1    442666
Name: Defect, dtype: int64


In [ ]:
model9=LGBMClassifier(colsample_bytree=0.5177908906249942,
               learning_rate=0.21034746012122046, max_bin=1023,
               min_child_samples=19, n_estimators=968, num_leaves=331,
               reg_alpha=0.026810451163601555, reg_lambda=0.026723380640834737,
               verbose=-1,n_jobs=-1)

model9.fit(X_sm1,Y_sm1)
pred=model9.predict(x_test)
print("mape:",mape(y_test,pred))
print("individual f1_score:",f1_score(y_test,pred,average=None))
print("weighted f1_score:",f1_score(y_test,pred,average="weighted"))
print("confusion_matrix:")
print(confusion_matrix(pred,y_test))
print("classification_report:")
print(classification_report(pred,y_test))

mape: 8636641862816.369
individual f1_score: [0.99660047 0.44       0.        ]
weighted f1_score: 0.9920237122999509
confusion_matrix:
[[49104   183    70]
 [   69    99     0]
 [   13     0     0]]
classification_report:
              precision    recall  f1-score   support

           0       1.00      0.99      1.00     49357
           1       0.35      0.59      0.44       168
           2       0.00      0.00      0.00        13

    accuracy                           0.99     49538
   macro avg       0.45      0.53      0.48     49538
weighted avg       1.00      0.99      0.99     49538



FITTING THE WHOLE DATASET IN MODEL 10 to test the final test data on the date of competition

In [ ]:
from imblearn.over_sampling import BorderlineSMOTE
smote=BorderlineSMOTE(sampling_strategy="not majority",n_jobs=-1,k_neighbors=1,m_neighbors=2,kind="borderline-2",random_state=42)
X_sm2,Y_sm2=smote.fit_resample(X1,Y1)
print(Y_sm2.value_counts())

0    491853
1    491853
2    491853
Name: Defect, dtype: int64


In [ ]:
model10=LGBMClassifier(colsample_bytree=0.5177908906249942,
               learning_rate=0.21034746012122046, max_bin=1023,
               min_child_samples=19, n_estimators=968, num_leaves=331,
               reg_alpha=0.026810451163601555, reg_lambda=0.026723380640834737,
               verbose=-1,n_jobs=-1)

model10.fit(X_sm2,Y_sm2)
pred=model10.predict(x_test)
print("mape:",mape(y_test,pred))
print("individual f1_score:",f1_score(y_test,pred,average=None))
print("weighted f1_score:",f1_score(y_test,pred,average="weighted"))
print("confusion_matrix:")
print(confusion_matrix(pred,y_test))
print("classification_report:")
print(classification_report(pred,y_test))

mape: 6.055957043078041e-05
individual f1_score: [0.9999695  1.         0.97810219]
weighted f1_score: 0.999938778291978
confusion_matrix:
[[49186     0     3]
 [    0   282     0]
 [    0     0    67]]
classification_report:
              precision    recall  f1-score   support

           0       1.00      1.00      1.00     49189
           1       1.00      1.00      1.00       282
           2       0.96      1.00      0.98        67

    accuracy                           1.00     49538
   macro avg       0.99      1.00      0.99     49538
weighted avg       1.00      1.00      1.00     49538



In [2]:
from joblib import Parallel, delayed
import joblib
  
  
# # Save the model as a pickle in a file
# joblib.dump(model8, '/content/drive/MyDrive/model weights/model8.pkl')
# joblib.dump(model9, '/content/drive/MyDrive/model weights/model9.pkl')
# joblib.dump(model10, '/content/drive/MyDrive/model weights/model10.pkl')

  
# Load the model from the file
# knn_from_joblib = joblib.load('filename.pkl')
  
# # Use the loaded model to make predictions
# knn_from_joblib.predict(X_test)

In [3]:
# Load the model from the file
mod = joblib.load('/content/model10.pkl')
  
# # Use the loaded model to make predictions

# pred=mod.predict(x_test)
# print("mape:",mape(y_test,pred))
# print("individual f1_score:",f1_score(y_test,pred,average=None))
# print("weighted f1_score:",f1_score(y_test,pred,average="weighted"))
# print("confusion_matrix:")
# print(confusion_matrix(pred,y_test))
# print("classification_report:")
# print(classification_report(pred,y_test))

In [11]:
testfile=pd.read_csv("/content/CBM_WM6undefined_Oct_30.csv", on_bad_lines='skip')
testfile.head()

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (0,1,4) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


,Company Number,Employee,Machine,Production,OrderOperation No,Date,Time,Current,humidity,temperature,flow,Job Temp,Voltage,Defect
0,undefined,undefined,undefined,undefined,undefined,10-11-2022,7:37:8:730,39.15000,57,23,0.093,30.7,11.585,No defect
1,undefined,undefined,undefined,undefined,undefined,10-11-2022,7:37:9:231,39.15000,57,23,0.093,30.7,11.585,No defect
2,undefined,undefined,undefined,undefined,undefined,10-11-2022,7:37:9:731,33.28125,57,23,0.000,29.4,10.087,No defect
3,undefined,undefined,undefined,undefined,undefined,10-11-2022,7:37:10:231,33.28125,57,23,0.000,29.4,10.087,No defect
4,undefined,undefined,undefined,undefined,undefined,10-11-2022,7:37:10:733,17.83125,57,23,0.125,30.0,9.773,No defect


testfilepreprocessing


In [13]:
testfile.columns

Index(['Company Number', 'Employee', 'Machine', 'Production',
       'OrderOperation No', 'Date', 'Time', 'Current', 'humidity',
       'temperature', 'flow', 'Job Temp', 'Voltage', 'Defect'],
      dtype='object')

In [29]:
df3=testfile.copy()
df3=df3.drop(0)
df3=df3.drop(["Date","Time",'Machine',"OrderOperation No","Production",],axis=1)

#encoding DEFECT column
df3.Defect=df3.Defect.replace({'No Defect':0,"No defect":0,'Tungsten Inclusion':1, 'No Defect73':0,
       'Tungsten Inclusion72':1, 'No Defect72':0, 'No Defect77':0,
       'No Defect79':0, 'No Defect80':0, 'Porosity':2, 'No Defect78':0,
       'No Defect76':0, 'No Defect0':0, 'No Defect82':0, 'No Defect85':0,
       'No Defect84':0, 'No Defect83':0, 'No Defect81':0, 'No Defect75':0,
       'No Defect86':0, 'Porosity86':2, 'No Defect87':0, 'Tungsten Inclusion78':1,
       'No Defect74':0, 'No Defect99':0, 'Porosity75':2, 'Tungsten Inclusion77':1,
       'No Defect71':0, 'No Defect40':0, 'Porosity80':2, 'No Defect70':0,
       'No Defect68':0, 'No Defect90':0, 'Tungsten Inclusion80':1,
       'No Defect69':0, 'No Defect67':0, 'No Defect63':0, 'No Defect64':0,
       'No Defect65':0, 'No Defect62':0, 'No Defect61':0, 'No Defect60':0,
       'No Defect59':0, 'Porosity76':2, 'No Defect66':0, 'Tungsten Inclusion71':1,
       'No Defect88':0, 'Tungsten Inclusion67':1, 'Tungsten Inclusion75':1,
       'Tungsten Inclusion76':1, 'Porosity71':2, 'No Defect58':0, 'No Defect57':0})

#encoding PRODUCTION column:
# df3.Production=df.Production.replace({'E15002965':1, 'E15002966':2 ,'-':4,
#        15003062:5, 'E15003220':6, 'E15002881':7, 'E10002436':8, 'E50006711':9,
#        'E15003200':10, 'E15003202':11, 'E15003056':12, 'E15002963':13, 'E10002437':14,
#        'E15003219':15, 'E15003201':16, 'Test coopan':17, -15003217:18, -150033027:19})

#converting object type to numeric type
cols=['Employee','Current',
       'humidity', 'temperature', 'flow', 'Job Temp', 'Voltage']
df3[cols] = df3[cols].apply(pd.to_numeric, errors='coerce')




df3=df3.fillna(method="ffill")

making new features from pre existing features in testfile

In [30]:
df3["f2"]=np.log(df3.Voltage**0.9/df3["Job Temp"]**0.1)
df3["f1"]=np.log(df3.Voltage*0.9499-df3["Job Temp"]**0.1)+df3.f2-np.log10(df3.f2**25)
df3["f3"]=np.log(df3.Voltage*0.9499-df3["Job Temp"]**0.1) -np.log(df3.Current**0.004)
df3["f4"]=(df3["Job Temp"]<145).astype("int")
# df3=df3.drop(["Defect","Production","Order Operation No"],axis=1)
df3=df3.replace([np.inf, -np.inf], 0)

/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: divide by zero encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log
  result = getattr(ufunc, method)(*inputs, **kwargs)
/usr/local/lib/python3.8/dist-packages/pandas/core/arraylike.py:364: RuntimeWarning: invalid value encountered in log10
  result = getattr(ufunc, method)(*inputs, **kwargs)


In [31]:
y_test=df3["Defect"]
x_test=df3.drop(["Defect","Company Number"],axis=1)


In [32]:
x_test.isnull().any()

Employee        True
Current        False
humidity       False
temperature    False
flow           False
Job Temp       False
Voltage        False
f2              True
f1              True
f3              True
f4             False
dtype: bool

In [33]:
df3.Defect.unique()

array([0, 1])

In [34]:
pred=mod.predict(x_test)
print("mape:",mape(y_test,pred))
print("individual f1_score:",f1_score(y_test,pred,average=None))
print("weighted f1_score:",f1_score(y_test,pred,average="weighted"))
print("confusion_matrix:")
print(confusion_matrix(pred,y_test))
print("classification_report:")
print(classification_report(pred,y_test))

mape: 116581801369726.2
individual f1_score: [0.9817959  0.30735405 0.        ]
weighted f1_score: 0.9698776253536181
confusion_matrix:
[[325268   3352      0]
 [  8619   2656      0]
 [    91      0      0]]
classification_report:


/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))


              precision    recall  f1-score   support

           0       0.97      0.99      0.98    328620
           1       0.44      0.24      0.31     11275
           2       0.00      0.00      0.00        91

    accuracy                           0.96    339986
   macro avg       0.47      0.41      0.43    339986
weighted avg       0.96      0.96      0.96    339986



/usr/local/lib/python3.8/dist-packages/sklearn/metrics/_classification.py:1318: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
